[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.leafmap.org/lab/index.html?path=notebooks/93_maplibre_pmtiles.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/leafmap/blob/master/docs/notebooks/93_maplibre_pmtiles.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeos/leafmap/HEAD)


**Visualizing PMTiles with Leafmap and MapLibre**

[PMTiles](https://github.com/protomaps/PMTiles) is a single-file archive format for tiled data. A PMTiles archive can be hosted on a commodity storage platform such as S3, and enables low-cost, zero-maintenance map applications that are "serverless" - free of a custom tile backend or third party provider.

In [ ]:
# %pip install -U "leafmap[maplibre]" pmtiles

In [ ]:
import leafmap.maplibregl as leafmap

## Remote PMTiles

### Source Cooperative

[Google-Microsoft Open Buildings - combined by VIDA](https://beta.source.coop/repositories/vida/google-microsoft-open-buildings/description)

In [ ]:
url = "https://data.source.coop/vida/google-microsoft-open-buildings/pmtiles/go_ms_building_footprints.pmtiles"
metadata = leafmap.pmtiles_metadata(url)
print(f"layer names: {metadata['layer_names']}")
print(f"bounds: {metadata['bounds']}")

In [ ]:
m = leafmap.Map(center=[0, 20], zoom=2, height="800px")
m.add_basemap("Esri.WorldImagery", visible=False)

style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "buildings",
            "source": "example_source",
            "source-layer": "building_footprints",
            "type": "fill",
            "paint": {"fill-color": "#3388ff", "fill-opacity": 0.5},
        },
    ],
}

# style = leafmap.pmtiles_style(url)  # Use default style

m.add_pmtiles(
    url,
    style=style,
    visible=True,
    opacity=1.0,
    tooltip=True,
)
m

In [ ]:
m.layer_interact()

![](https://i.imgur.com/rH0iLgc.png)

## Local PMTiles

tippecanoe is required to convert vector data to pmtiles. Install it with `conda install -c conda-forge tippecanoe`.

Download [building footprints](https://github.com/opengeos/open-data/blob/main/datasets/libya/Derna_buildings.geojson) of Derna, Libya.

In [ ]:
url = "https://raw.githubusercontent.com/opengeos/open-data/main/datasets/libya/Derna_buildings.geojson"
leafmap.download_file(url, "buildings.geojson")

Convert vector to PMTiles.

In [ ]:
pmtiles = "buildings.pmtiles"
leafmap.geojson_to_pmtiles(
    "buildings.geojson", pmtiles, layer_name="buildings", overwrite=True, quiet=True
)

Start a HTTP Sever

In [ ]:
leafmap.start_server(port=8000)

In [ ]:
url = f"http://127.0.0.1:8000/{pmtiles}"
# leafmap.pmtiles_metadata(url)

Display the PMTiles on the map.

In [ ]:
m = leafmap.Map()
m.add_basemap("Esri.WorldImagery")
style = {
    "version": 8,
    "sources": {
        "example_source": {
            "type": "vector",
            "url": "pmtiles://" + url,
            "attribution": "PMTiles",
        }
    },
    "layers": [
        {
            "id": "buildings",
            "source": "example_source",
            "source-layer": "buildings",
            "type": "fill",
            "paint": {"fill-color": "#3388ff", "fill-opacity": 0.5},
        },
    ],
}

# style = leafmap.pmtiles_style(url)  # Use default style

m.add_pmtiles(
    url,
    style=style,
    visible=True,
    opacity=0.8,
    tooltip=True,
)
m

In [ ]:
m.layer_interact()

![](https://i.imgur.com/PnYhph1.jpeg)